In [ ]:
!pip install torch torchvision pillow gradio transformers tqdm


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from PIL import Image
import gradio as gr
from transformers import pipeline
import os


In [ ]:
# If your ZIP is in the default Colab directory (e.g., '/content')
import os

zip_path = "/content/drive/MyDrive/CDDM-dataset.zip"  # Change this if needed
extract_path = "/content/CDDM"          # Target directory for extraction

# Create extract directory if not exists
os.makedirs(extract_path, exist_ok=True)

# Unzip dataset using the unzip command
!unzip -q "$zip_path" -d "$extract_path"

print("Dataset unzipped to:", extract_path)

replace /content/CDDM/images/Apple,Alternaria Blotch/plant_69422.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
A



Dataset unzipped to: /content/CDDM


In [ ]:
DATA_DIR = "/content/CDDM"
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = ImageFolder(DATA_DIR, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

print(f"Found {len(train_dataset)} images across {len(train_dataset.classes)} classes.")




Found 137007 images across 1 classes.


In [ ]:
!ls "/content/CDDM"

Crop_Disease_train_llava.json  images


In [ ]:
#Data Transforms and Loader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch

# Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# ✅ Correct dataset path
DATA_DIR = "/content/CDDM"

# Create dataset and dataloader
train_dataset = ImageFolder(DATA_DIR, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

print(f"✅ Loaded {len(train_dataset)} training images across {len(train_dataset.classes)} classes.")
print("Classes:", train_dataset.classes)


✅ Loaded 137007 training images across 1 classes.
Classes: ['images']


In [ ]:
#6.Model Definition
#Use a simple CNN or pretrained model (here, ResNet18):

import torch
import torch.nn as nn
from torchvision import models

num_classes = len(train_dataset.classes)
model = models.resnet18(weights=None)  # train from scratch
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")



Epoch 1, Loss: 0.0
Epoch 2, Loss: 0.0
Epoch 3, Loss: 0.0
Epoch 4, Loss: 0.0
Epoch 5, Loss: 0.0


In [ ]:
torch.save(model.state_dict(), "cddm_leaf_model.pth")


In [ ]:
model.load_state_dict(torch.load("cddm_leaf_model.pth", map_location=device))
model.eval()

class_names = train_dataset.classes


In [ ]:
!pip install gradio pillow


In [ ]:
import json
import gradio as gr
from PIL import Image
import os


In [ ]:
# Path to your conversational dataset (update as needed)
json_path = "/content/drive/MyDrive/Crop_Disease_train_llava.json"

with open(json_path, 'r') as f:
    all_cases = json.load(f)

# Build a dictionary: image path -> conversations list
image_convo_dict = {
    item["image"]: item["conversations"]
    for item in all_cases
}


In [ ]:
def extract_plant_name(image_path):
    # "/dataset/images/Apple,Alternaria Blotch/plant_74609.jpg"
    base = image_path.split("/")[-2]     # "Apple,Alternaria Blotch"
    plant = base.split(",")[0]           # "Apple"
    return plant

In [ ]:
def get_disease_info(image_path):
    convos = image_convo_dict.get(image_path, [])
    for i, turn in enumerate(convos):
        if turn["from"] == "human" and "disease" in turn["value"].lower():
            if i+1 < len(convos) and convos[i+1]["from"] == "gpt":
                return convos[i+1]["value"]
    for turn in convos:
        if turn["from"] == "gpt" and "affect" in turn["value"].lower():
            return turn["value"]
    return "Disease information not found."

In [ ]:
def get_prevention_info(image_path):
    convos = image_convo_dict.get(image_path, [])
    for i, turn in enumerate(convos):
        if turn["from"] == "human" and any(k in turn["value"].lower() for k in ["prevent", "cure", "treat", "remedy"]):
            if i+1 < len(convos) and convos[i+1]["from"] == "gpt":
                return convos[i+1]["value"]
    return "No specific prevention advice in this conversation."

In [ ]:
def chatbot(image, query=""):
    # Prompt if no image
    if image is None:
        return "Please upload a leaf image to start the diagnosis chat."

    # Get the filename for lookup in your dataset
    image_path = None
    if hasattr(image, "name"):
        image_path = os.path.basename(image.name)
    else:
        return "Image format not recognized. Please upload a valid file."

    found_key = None
    for key in image_convo_dict:
        if image_path in key:
            found_key = key
            break

    if found_key is None:
        return "Image not found in the dataset. Please check if your image is part of the dataset!"

    disease_info = get_disease_info(found_key)
    plant_name = extract_plant_name(found_key)

    # Fresh/healthy leaf logic
    if any(txt in disease_info.lower() for txt in ["healthy", "no disease", "normal", "fresh"]):
        return f"Plant: {plant_name}\nNo disease detected in this leaf."

    # Diseased leaf logic
    if not query:
        return f"Plant: {plant_name}\nDisease detected: {disease_info}\nType your question below to get prevention advice."
    else:
        prevention_info = get_prevention_info(found_key)
        return f"Plant: {plant_name}\nDisease detected: {disease_info}\nPrevention advice: {prevention_info}"


In [ ]:
def chatbot(image, query=""):
    if image is None or not os.path.isfile(image):
        return "Please upload a leaf image to start the diagnosis chat."
    image_path = os.path.basename(image)

    found_key = None
    for key in image_convo_dict:
        if image_path in key:
            found_key = key
            break

    if found_key is None:
        return "Image not found in the dataset. Please check if your image is part of the dataset!"

    disease_info = get_disease_info(found_key)
    plant_name = extract_plant_name(found_key)

    if any(txt in disease_info.lower() for txt in ["healthy", "no disease", "normal", "fresh"]):
        return f"Plant: {plant_name}\nNo disease detected in this leaf."
    if not query:
        return f"Plant: {plant_name}\nDisease detected: {disease_info}\nType your question below to get prevention advice."
    else:
        prevention_info = get_prevention_info(found_key)
        return f"Plant: {plant_name}\nDisease detected: {disease_info}\nPrevention advice: {prevention_info}"


In [ ]:
gr.Interface(
    fn=chatbot,
    inputs=[
        gr.Image(type="filepath", label="Upload Leaf Image"),   # Use a descriptive label
        gr.Textbox(label="Ask about prevention")                # Remove `optional=True`
    ],
    outputs="text",
    title="Leaf Disease Diagnosis and Prevention Chatbot",
    description="Please upload a leaf image to start. If a disease is detected, you can ask for prevention advice."
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7f539b300d8269c1e5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
